# RotateProxies doc
This class manages the proxies rotation in HTTPS requests during scraping.
____
Requirements:

In [1]:
from bs4 import BeautifulSoup
from itertools import cycle
from requests import get

The instance of the class `RotateProxies` has only as attribute an iterator `proxies` which is initialized with the instance thanks to the class method *collectProxies()*.

In [ ]:
class RotateProxies:
    def __init__(self):
        self.proxies = self.collectProxies()

The **collectProxies()** method collects proxies (IP:Port) from the [Free Proxy List](https://free-proxy-list.net/) site, only those available for HTTPS requests.

It returns a cyclic iterator.  
This method is called when initializing an instance.

In [ ]:
class RotateProxies:
    ...
    def collectProxies(self):
        url = "https://free-proxy-list.net/"
        table = BeautifulSoup(get(url).text, 'html.parser').find(
            'table', attrs={'id': 'proxylisttable'})

        proxies = cycle([{
            "https":
            "{}:{}".format(
                row.find_all('td')[0].string,
                row.find_all('td')[1].string)
        } for row in table.find_all('tr') if len(row.find_all('td')) > 1
                         and row.find_all('td')[6].string == "yes"])

        return proxies

The **next()** method is the iterator protocol.

But before sending the next proxy, it tests its validity.  
If it's not valid, it tests the next one and so on.

In [2]:
 class RotateProxies:
    ...
    def next(self):
        test_url = 'https://httpbin.org/ip'
        proxy = None
        while True:
            proxy = next(self.proxies)
            try:
                test = get(test_url, proxies = proxy)
            except:
                continue
            else:
                break
        return proxy

# Example of use
An available random proxy.

In [3]:
p = RotateProxies()
p.next()

{'https': '101.51.241.253:8080'}